<h1>Language Model</h1>

In [15]:
import timeit

import os
import nltk
import re
import pandas as pd
import string
import ast
from itertools import islice

from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

def join_tags(tokens):
    for i in enumerate(tokens):
        hit = 0
        index = i[0]
        y = i[1]
        width = len(y)
        
        for x in y:
            if x is '-':
                #save = tokens[index][hit+1:]
                #tokens[index] = tokens[index][0:hit] + ''
                #tokens[index+1] = save
                #print(save)
                #print(tokens[index])
                tokens[index] = tokens[index][0:hit] + ' ' + tokens[index][hit+1:]
            hit += 1
        
        if tokens[index] is ' ':
            tokens[index] = ''
        
        if tokens[index] == '  ':
            tokens[index] = ''
        
        if y[(width-1):width] is '.':
            tokens[index] = y[0:(width-1)]
        
        if tokens[index][0:1] is '-' or tokens[index][1:2] is '-':
            tokens[index] = ''
        
        if tokens[index][-2:-1] is "'":
            tokens[index] = tokens[index][0:-2]
        
        if  tokens[index][:1] is "'" :
            tokens[index] = tokens[index][1:]
        
        if tokens[index] is "s" or tokens[index] is "re":
            tokens[index-1] = tokens[index-1]
            tokens[index] = ''
        
        if tokens[index] is '<' and tokens[index+2] is '>':
            tokens[index+2] = '<'+ tokens[index+1]+'>'
            tokens[index] =''
            tokens[index+1] = ''
            
        if tokens[index] is '&' and tokens[index+2] is ';':
            tokens[index+2] = ''
            tokens[index] = ''
            tokens[index+1] = ''
            
        if tokens[index] is '(' or tokens[index] is ')':
            tokens[index] = ''
            
        if tokens[index][0:1] is ':':
            tokens[index] = ''
        
        if tokens[index][0:1] is '.' and tokens[index][1:2] is '.':
            tokens[index] = ''

        if y[(width-2):(width-1)] == '.' and y[(width-1):(width)] == '0':
            tokens[index] = y[0:(width-2)]
        
    tokens_clean = [x for x in tokens if x != '' and x not in ".,!?'" and x not in '<>']
        
    return tokens_clean

def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

def guess_date(string):
    for fmt in ["%Y/%m/%d", "%d-%m-%Y", "%Y%m%d", "%d-%b-%y", "%d-%m-%y"]:
        try:
            return datetime.datetime.strptime(string, fmt).date()
        except ValueError:
            continue
    raise ValueError(string)

def preprocess(text_process):
    text_process = text_process.lower()
    tokens = word_tokenize(text_process)
    tokens_clean = join_tags(tokens)
    stop_words = set(stopwords.words('english'))
    filtered_words = [w for w in tokens_clean if not w in stop_words]
    ps = PorterStemmer()
    stemmed_words = [ps.stem(w) for w in filtered_words]
    lemmatizer = WordNetLemmatizer()
    lemmatized_words = [lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in stemmed_words]
    stemmed_text = ' '.join(stemmed_words)
    lemmatized_text = ' '.join(lemmatized_words)
    #clean_text = lemmatized_text
    clean_text = lemmatized_text
    clean_words = sorted(list([lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in stemmed_words]))
    clean_text = '<?xml version="1.0" encoding="UTF-8"?><root>' + clean_text + '</root>'
    return clean_text, clean_words

def


def language_model(query, limit, lambd):
    clean_query_text, clean_query_words = preprocess(query)
#     lambd = 1/4
#     limit = 10
    result_dict = list()

    df_doc_freq = pd.read_csv('Document Frequencies.csv')
    df = pd.read_csv('Frequencies.csv')

    global_freq = int(open("global_frequency.txt", "r").read())
    print(global_freq)

    start = timeit.default_timer()
    for i in df_doc_freq.iterrows():
        local_freq = i[1]['Frequency']
        doc_number = i[1]['Document']
        doc_number = str(doc_number).zfill(3)
        result = 1
        for j in query.split():
            entries = df.loc[df['Term'] == j]['Documents'].values[0]
            entries = entries.split(', ',1)
            entries[0] = entries[0].replace("[", "")
            entries[1] = entries[1].replace(']', '')
            entries[1] = ast.literal_eval(entries[1])
            if doc_number in entries[1].keys():
                prob = (float(entries[1][doc_number])/float(local_freq)*lambd) + (float(entries[0])/float(global_freq)*(1-lambd))
            else:
                prob = (float(entries[0])/float(global_freq)*(1-lambd))
            result *= prob
        result_dict.append((doc_number, result))
    result_sorted = sorted(result_dict, key=lambda x: x[1], reverse=True)
    stop = timeit.default_timer()
    return result_sorted[:limit], stop-start

In [16]:
language_model('1991', 10, 1/2)

50615


([('031', 0.020922239783990253),
  ('190', 0.010505573117323586),
  ('301', 0.007441847627127508),
  ('105', 0.006582412944163413),
  ('070', 0.005243545625914651),
  ('422', 0.004474871362937621),
  ('466', 0.0039648754429049814),
  ('061', 0.002720485398025341),
  ('001', 8.890645065691988e-05),
  ('002', 8.890645065691988e-05)],
 0.8783825000000434)

In [23]:
import datetime

def guess_date(string):
    for fmt in ["%Y/%m/%d", "%d-%m-%Y", "%Y%m%d", "%d-%b-%y", "%d-%m-%y"]:
        try:
            return datetime.datetime.strptime(string, fmt).date()
        except ValueError:
            continue
    raise ValueError(string)
    
samples = "2017/5/23", "22-04-2015", "20150504", "9-Mar-87", "09-03-87"

for sample in samples:
    print(guess_date(sample))

2017-05-23
2015-04-22
2015-05-04
1987-03-09
1987-03-09


<h1>TF IDF</h1>

In [43]:
import os
import nltk
import re
import pandas as pd
import string
import ast
from itertools import islice
import math


df_tfidf = pd.read_csv('Konstruksi Indeks.csv')
doc = df_tfidf['Documents']
term = df_tfidf['Term']

dict_tfidf = dict()
dict_doc = dict()
list_doc = list()
dic = dict()
max_range = len(term)
N = 500
for i in range (0,max_range):
    start = 2
    end = 5
    count = 1
    for y in range (0,int(len(doc[i])/7)):
        if(int(len(doc[i])/7)!=1):
            if(doc[i][start:end] != doc[i][(start-7):(end-7)]):
                dict_doc[term[i]+" "+doc[i][start:end]] = count
                dic[doc[i][start:end]] = count
            else:
                dict_doc[term[i]+" "+doc[i][start:end]] = count
                dic[doc[i][start:end]] = count
                count+=1
        else :
            dict_doc[term[i]+" "+doc[i][start:end]] = count
            dic[doc[i][start:end]] = count
        start = start+7
        end = end+7
        dict_tfidf[term[i]] = count

def tfidf(query):
    text, data = preprocess(query)
    z=1
    count = 1
    N = 500+1

    dict_w = dict()
    dict_f = dict()
    dict_tf = dict()
    dict_idf = dict()
    dict_dj = dict()
    dict_sim = dict()

    for q in data:
        for i in range (0,max_range):
            ni = 0
            start = 2
            end = 5
            count = 1
            if (term[i] == q):
                for y in range (0,int(len(doc[i])/7)):
                    if(int(len(doc[i])/7)!=1):
                        if(doc[i][start:end] != doc[i][(start-7):(end-7)]):
                            ni+=1
                            dict_f['f',doc[i][start:end]]=count
                        else:
                            dict_f['f',doc[i][start:end]]=count
                            count+=1
                            ni=1
                    else:
                        dict_f['f',doc[i][start:end]]=count
                        ni+=1

                    start = start+7
                    end = end+7

                dict_idf[q] = math.log((1+N/ni),10)


    for key in dict_f:
        dict_tf['tf',z] = 1+math.log(dict_f[key],10)
        z+=1

    z=1

    for key in dict_tf:
        for key2 in dict_idf:
            dict_w['w',z] = dict_tf[key]*dict_idf[key2]
            z+=1

    z=1

    for key in dict_w:
        dict_dj['dj',z] = math.sqrt(pow(dict_w[key],2))
        z+=1

    z=1
    for key in dict_dj:
        for key2 in dict_w:
            dict_sim["d",z] = dict_w[key2]/dict_dj[key]
            z+=1
            break
    stop = timeit.default_timer()
#     print(data)
#     print("N : ",N)
#     print("f : ",dict_f)
#     print("Tf : ",dict_tf)
#     print("IDF : ",dict_idf)
#     print("W : ", dict_w)
#     print("Panjang : ", dict_dj)
#     print("Sim : ",dict_sim)
    
    result = [(i[1], j)for i,j in dict_f.items()]
    
    return result, stop-start 

In [44]:
tfidf("1991")

([('031', 1),
  ('061', 1),
  ('070', 1),
  ('105', 1),
  ('190', 1),
  ('301', 1),
  ('422', 1),
  ('466', 1)],
 2992.5141184)